# Notebook to demonstrate the use of some functions defined in analysis_helpers.py
#### Brandon Chan August 2020

### Import packages

In [13]:
import sys
sys.path.insert(0,"../src")

import pandas as pd
import numpy as np
from model_helpers import check_excel_file
from analysis_helpers import apply_new_costs, apply_new_utilities, get_state_mappings

### Read in model specifications and existing population array

In [2]:
input_file = pd.ExcelFile('../model_specifications/test_parameters_base.xlsx') # read in excel file
check_excel_file(input_file) # run checks on file formatting and specification of model

# Read in relevant speadsheet in the file
costs_df = pd.read_excel(input_file, 'costs')
utilities_df = pd.read_excel(input_file, 'utilities')
specification_df = pd.read_excel(input_file, 'specification', header=None, index_col=0)

state_mapping = get_state_mappings('../model_specifications/test_parameters_base.xlsx')

pop_base = np.load('../model_outputs/test_base_20-07-2020_population.npy')

In [3]:
costs_df

,state,type,cost,cost_variance
0,treatment,static,350,0
1,remission,static,0,0
2,death,static,0,0


In [4]:
utilities_df

,state,type,utility,utility_variance
0,treatment,static,0.85,0
1,remission,static,1.00,0
2,death,static,0.00,0


In [5]:
specification_df

,1
0,
max_iterations,1000
time_horizon,1
cycle_length,60
discount_rate,0.015
name_start_state,treatment


### Applying new costs and utilities to existing population arrays
Could be used for quick and dirty sensitivity analysis?

#### 2 Methods of doing this:
* 1) Read in a new spreadsheet with new defined costs and/or utilities
* 2) Modify the dataframes in-code to specify new values (recommended, as preserving the format and values of the original model helps bookkeep better) 

In [6]:
# breakdown:
# dataframe.loc[row, column] = new_value
# in this example, the row is selected by a conditional. As such we want to select the row where the value of 'state' is equal to 'treatment'
# futher, we want to target the cost column.

costs_df.loc[costs_df.state == 'treatment', 'cost'] = 700
costs_df

,state,type,cost,cost_variance
0,treatment,static,700,0
1,remission,static,0,0
2,death,static,0,0


In [12]:
# Function call is as follows: apply_new_costs(population_array, discount_rate, cycle_length, state_mapping, costs)
new_cost_base = apply_new_costs(pop_base, 0.015, 60, state_mapping, costs_df)
new_cost_base[0,:,:]

array([[700.        , 252.92523445,  91.38739174,  33.02025355,
         11.93093624,   4.31090692],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ]])

#### Check that the recalculated costs are double the original (as intended)

In [10]:
cost_base = np.load('../model_outputs/test_base_20-07-2020_costs.npy')
cost_base[0,:,:]

array([[350.        , 126.46261722,  45.69369587,  16.51012677,
          5.96546812,   2.15545346],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ]])